In [18]:
import pandas as pd

In [19]:
df=pd.read_csv("data/gemstone.csv")
df=df.drop(labels=['id'],axis=1)


In [20]:
#seggragating the numercal and categorical columns
numerical_columns=df.columns[df.dtypes!='object']
categorical_columns=df.columns[df.dtypes=='object']
print("numerical column",numerical_columns)
print("categorical_columns",categorical_columns)

numerical column Index(['carat', 'depth', 'table', 'x', 'y', 'z', 'price'], dtype='object')
categorical_columns Index(['cut', 'color', 'clarity'], dtype='object')


In [21]:
df['cut'].value_counts()

cut
Ideal        92454
Premium      49910
Very Good    37566
Good         11622
Fair          2021
Name: count, dtype: int64

In [22]:
df['color'].value_counts()

color
G    44391
E    35869
F    34258
H    30799
D    24286
I    17514
J     6456
Name: count, dtype: int64

In [23]:
df['clarity'].value_counts()

clarity
SI1     53272
VS2     48027
VS1     30669
SI2     30484
VVS2    15762
VVS1    10628
IF       4219
I1        512
Name: count, dtype: int64

In [24]:
#independent and dependent features
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [25]:
Y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [26]:
categorical_columns=X.select_dtypes(include='object').columns
numerical_columns=X.select_dtypes(exclude='object').columns

In [27]:
#define the ranking for each ordinal variable
cut_categories=['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_categories=['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_categories=['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [32]:
#Here we have automated the whole preprocessing step
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler #Handling feature scaling
from sklearn.preprocessing import OrdinalEncoder #ordinal encoding
#pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 


In [33]:
#NUmerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
#Categorical Pipeline 
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_columns),
    ('cat_pipeline',cat_pipeline,categorical_columns)
])

In [34]:
#Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [35]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [36]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.652424,0.058753,1.605394
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-0.381506,1.123070,-0.079246
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-1.415437,1.123070,-1.202339
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.652424,-1.537723,2.166941
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-0.381506,1.655229,0.482301


In [37]:
#Model training

from sklearn.linear_model import LinearRegression,Lasso, Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [38]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [39]:
regression.coef_

array([[ 6538.82211476,  -232.4061404 ,  -177.05602534, -2090.53215704,
         -556.05883268,   -44.19321465,   -10.72128479,  -181.811724  ,
          121.17907723]])

In [40]:
regression.intercept_

array([3976.8787389])

In [41]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae, mse, rmse, r2_square

In [45]:
#train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
    
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae,mse,rmse,r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model Training Performance")
    print("RMSE",rmse)
    print("MAE",mae)
    print("R2_Score",r2_square*100)

    r2_list.append(r2_square)
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE 1196.8248815972495
MAE 723.6034569047442
R2_Score 91.1356057138532


Lasso
Model Training Performance
RMSE 1196.7936860863608
MAE 725.0550559822407
R2_Score 91.13606781270963


Ridge
Model Training Performance
RMSE 1196.8307856345255
MAE 723.6370749375668
R2_Score 91.13551825604013


Elasticnet
Model Training Performance
RMSE 1632.539414426596
MAE 1135.0835947511696
R2_Score 83.5064099195437


